In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats

In [4]:
# Files to Load
EV_Data = Path("Electric_Vehicle_Population_Data.csv")

# Read sneaker data and store into Pandas DataFrames
EV_df = pd.read_csv(EV_Data)
EV_df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5UXTA6C03N,King,Auburn,WA,98001,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,47,240226332,POINT (-122.2849393 47.3384055),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033029902
1,5YJ3E1EB0N,Thurston,Olympia,WA,98513,2022,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,2,205746894,POINT (-122.817545 46.98876),PUGET SOUND ENERGY INC,53067011624
2,5UXTA6C00N,Snohomish,Snohomish,WA,98296,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30,0,1,180458386,POINT (-122.15134 47.8851158),PUGET SOUND ENERGY INC,53061052122
3,1FMCU0LZ9N,Whitman,Pullman,WA,99163,2022,FORD,ESCAPE,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,9,200792707,POINT (-117.17912 46.730885),AVISTA CORP,53075000300
4,7SAYGDEE2N,Whitman,Pullman,WA,99163,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,9,193579760,POINT (-117.17912 46.730885),AVISTA CORP,53075000201
